In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import string
import operator
import pandas as pd
import numpy as np
import seaborn as sns

from collections import Counter
from functools import reduce

import matplotlib.pyplot as plt
plt.style.use('ggplot')

from probability.experiment import Experiment
from probability.new.joint_distribution import JointDistribution

In [2]:
ACCEPT_LETTERS = string.ascii_lowercase + ' '
FILE = './brazilian_tales_2017_1/LIFE_by_JM_Machado_de_Assis.txt'

ACCEPT_LETTERS, FILE

('abcdefghijklmnopqrstuvwxyz ',
 './brazilian_tales_2017_1/LIFE_by_JM_Machado_de_Assis.txt')

# Questão 1

## a) Calculate and plot the probability distribution $P(x)$ over the 27 letters $x$.


In [27]:
import operator
from functools import reduce
import re

def read_file(file):
    with open(file, 'r') as f:
        return f.read().lower()

def read_file_regex(file):
    f = read_file(file)
    regex = r'[^{}]'.format(ACCEPT_LETTERS)
    
    resultado = re.sub(regex, ' ', f)
    
    total_anterior, total_corrente = len(resultado) + 1, len(resultado)
    while total_anterior > total_corrente:
        resultado = resultado.replace('  ', ' ')
        total_anterior, total_corrente = total_corrente, len(resultado)
    
    return resultado

text = read_file_regex(FILE)
read_file(FILE)[:250], read_file_regex(FILE)[:250]

('life\n\nby joaquim maria machado de assis\n\n\n    end of time. ahasverus, seated upon a rock, gazes for a long while\n    upon the horizon, athwart which wing two eagles, crossing each\n    other in their path. he meditates, then falls into a doze. the day',
 'life by joaquim maria machado de assis end of time ahasverus seated upon a rock gazes for a long while upon the horizon athwart which wing two eagles crossing each other in their path he meditates then falls into a doze the day wanes ahasverus i have')

In [41]:
def distribuicao_probabilistica(file):
    series = pd.Series(Counter(text))
    series.name = 'Letters'
    return JointDistribution.from_series(series)

P = distribuicao_probabilistica(text)

plt = P.plot()
plt.title('Item a - Distribuição probabilística\n{}'.format(plt.axes().get_title()))
plt.show()
#Experiment.from_counter(Counter(text), 'Letter')#.occurrences[0].to_series()

TypeError: Can't convert 'NoneType' object to str implicitly

## b) Calculate and plot the probability distribution $P(x, y)$ over the $27 x 27$ possible (ordered) bi-grams $xy$. Note: For this joint distribution, its two marginals $P(x)$ and $P(y)$ are identical.

In [ ]:
def distribuicao_probabilistica_digrafo(text):
    counter = Counter(zip(text[0:-1], text[1:]))
    
    return Experiment.from_counter(counter, ('X', 'Y')).calcule()

P = distribuicao_probabilistica_digrafo(text)
P.to_dataframe().head()

In [ ]:
P.plot()

## c) From this joint distribution, calculate and plot the conditional distribution: 

### $i)$ $P(y\ |\ x)$, the conditional distribution of the second letter $y$, given the fist letter $x$;

In [ ]:
X, Y = P.variables

P(Y | X).to_dataframe().head()

In [ ]:
axes = P(Y | X).plot()
axes.set_title('Item c - Distribuição probabilística condicional\n$P(y\ |\ x)$')

### $ii)$ $P(x\ |\ y)$, the conditional distribution of the first letter $x$, given the second letter $y$.

In [ ]:
P(X | Y).to_dataframe().head()

In [ ]:
P(X | Y).plot()